# k-means聚类算法

## 原理

### 步骤

- 求距离
    - result = np.sqrt(np.sum((x_train - i) ** 2, axis=1))  
      
      
- 寻找最小的k个距离
    - result_index = np.argsort(result)[:k]  
      
      
- 统计
    - votes = Counter(predict_result)

### 基础原理

距离相近的数据大概率拥有相同的特性  
  
  
欧拉距离:
    $$ d=\sqrt{ \sum_{i=1}^{n}(x_{i}^{a} - x_{i}^{b})^2} $$  
曼哈顿距离:  
    $$ d=\sum_{i=1}^{n}\left|(x_{i}^{a} - x_{i}^{b})\right| $$ 
明可夫距离:  
    $$ d={(\sum_{i=1}^{n}\left|x_{i}^{a} - x_{i}^{b}\right|^p}) ^\frac{1}{p}$$ 

### 考虑距离的优化

![](https://raw.githubusercontent.com/LinCheungS/PicGo_Image_Storage/master/2020/20200421103414.jpg)

### 优点与缺点

1. 优点  
    - 算法简单
    - 天然可以解决多分类问题  
      
      
2. 缺点
    - 效率低下(m个样本,n个特征,时间成本O(m*n))
    - 高度数据相关
    - 预测的结果不具有可解释性
    - 维数灾难
      - ![](https://raw.githubusercontent.com/LinCheungS/PicGo_Image_Storage/master/2020/20200421103428.jpg)

## 底层实现

In [1]:
import numpy as np
from collections import Counter

class KNN:
    def __init__(self, data):
        self.data = data

    def split_data(self, ratio):
        index = np.random.permutation(len(self.data.target))
        train_data_size = int(len(index) * ratio)
        index_train = index[:train_data_size]
        index_text = index[train_data_size:]
        x_train = self.data.data[index_train]
        y_train = self.data.target[index_train]
        x_text = self.data.data[index_text]
        y_text = self.data.target[index_text]
        return x_train, y_train, x_text, y_text

    def do_knn(self, x_train, y_train, x_text, y_text, k):
        final = np.array([])
        for i in x_text:
            result = np.sqrt(np.sum((x_train - i) ** 2, axis=1))
            result_index = np.argsort(result)[:k]
            predict_result = y_train[result_index]
            votes = Counter(predict_result)
            final_results = votes.most_common(1)[0][0]
            final = np.append(final, final_results)
        accuracy = sum(final == y_text) / len(y_text)
        return "准确率是:" + str(accuracy)

In [2]:
import sklearn.datasets as datasets
data = datasets.load_iris()
knn = KNN(data)
x_train, y_train, x_text, y_text = knn.split_data(0.8)
knn.do_knn(x_train, y_train, x_text, y_text,k=3)

'准确率是:0.9333333333333333'

## sklearn中的实现

### 通用流程

In [3]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

#加载数据
digits = datasets.load_digits()
x = digits.data
y = digits.target

#分离数据
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

#训练数据
knn= KNeighborsClassifier(n_neighbors=3,n_jobs=2)
knn.fit(X_train,y_train)
knn.score(X_test,y_test)

1.0

### 手动寻找参数p

In [4]:
best_p = 0.0
best_k = 0.0
best_accuracy = 0.0

for p in range(1, 20):

    # k的值必须大于0
    for k in range(1, 20):
        knn = KNeighborsClassifier(
            n_neighbors=k, n_jobs=2, weights="distance", p=p)
        knn.fit(X_train, y_train)
        accuracy = knn.score(X_test, y_test)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_p = p
            best_k = k
print("best_accuracy = {a}".format(a=best_accuracy))
print("best_p = {a}".format(a=best_p))
print("best_k = {a}".format(a=best_k))

best_accuracy = 1.0
best_p = 2
best_k = 3


### 网格搜索参数p

In [5]:
# (自带交叉验证)
param_grid = [
    {
        "weights": ["uniform", "distance"],
        "n_neighbors": [i for i in range(1, 10)],
        "p": [i for i in range(1, 10)]
    }
]

In [6]:
from sklearn.model_selection import GridSearchCV


knn = KNeighborsClassifier()
#传入算法与网格参数
grid_search = GridSearchCV(knn,param_grid,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)
# 打印最好的参数
grid_search.best_estimator_

Fitting 5 folds for each of 162 candidates, totalling 810 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 533 tasks      | elapsed:   51.2s
[Parallel(n_jobs=-1)]: Done 810 out of 810 | elapsed:  1.3min finished


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=5,
                     weights='uniform')

In [7]:
# 训练时的最好成绩
grid_search.best_score_

0.9867813588850174

In [8]:
# 最好的参数
grid_search.best_params_

{'n_neighbors': 1, 'p': 5, 'weights': 'uniform'}

In [9]:
# 把最高的模型赋值给new_knn_clf
new_knn_clf = grid_search.best_estimator_
new_knn_clf.score(X_test, y_test)

1.0

In [10]:
new_knn_clf.predict(X_test)

array([7, 1, 5, 0, 1, 2, 6, 5, 6, 5, 7, 4, 3, 4, 9, 0, 8, 5, 1, 7, 6, 3,
       3, 4, 5, 1, 7, 9, 7, 9, 8, 0, 9, 2, 2, 0, 2, 4, 3, 8, 9, 1, 0, 1,
       9, 0, 1, 2, 0, 0, 5, 7, 0, 7, 5, 1, 8, 4, 1, 4, 2, 6, 6, 9, 2, 6,
       3, 6, 6, 4, 6, 1, 2, 4, 5, 1, 2, 7, 6, 4, 1, 3, 0, 9, 7, 3, 8, 4,
       5, 1, 3, 2, 2, 3, 8, 9, 9, 5, 7, 7, 2, 9, 2, 4, 5, 5, 3, 0, 9, 4,
       2, 8, 3, 7, 5, 3, 6, 6, 1, 3, 3, 9, 7, 3, 2, 4, 7, 9, 0, 5, 2, 0,
       0, 3, 1, 8, 4, 4, 4, 6, 5, 8, 9, 9, 5, 1, 2, 7, 1, 7, 9, 1, 5, 5,
       2, 4, 2, 2, 8, 1, 5, 7, 5, 0, 9, 7, 2, 2, 8, 3, 2, 8, 0, 6, 2, 1,
       5, 5, 0, 7, 0, 4, 8, 1, 8, 6, 3, 5, 4, 8, 8, 5, 6, 7, 2, 4, 6, 9,
       9, 4, 8, 3, 9, 6, 0, 0, 9, 0, 4, 0, 6, 3, 6, 2, 7, 4, 8, 8, 7, 1,
       8, 1, 4, 2, 1, 3, 2, 8, 1, 4, 5, 6, 4, 3, 4, 9, 2, 7, 3, 5, 2, 1,
       1, 5, 5, 3, 3, 3, 4, 1, 6, 0, 1, 3, 9, 9, 5, 6, 2, 2, 9, 9, 3, 9,
       3, 8, 7, 9, 8, 0, 8, 3, 4, 8, 6, 8, 2, 8, 8, 7, 1, 2, 0, 6, 0, 8,
       1, 7, 5, 0, 3, 0, 0, 7, 9, 9, 2, 7, 3, 3, 7,